# Binary Split Decision Tree

In [5]:
import numpy as np
from collections import Counter

In [32]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature          # Feature index used for splitting
        self.threshold = threshold      # Value used for splitting
        self.left = left                # Left child node (<= threshold)
        self.right = right              # Right child node (> threshold)
        self.value = value              # If leaf node, holds class label
    
    def is_leaf_node(self):
        return self.value is not None

In [40]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=10, min_sample_split=2, n_features = 10):
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.n_features = n_features
        self.root = None
        
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._build_tree(X,y)
        
    def _build_tree(self, X, y, depth=0):
        n_sample, n_feats = X.shape
        n_labels = len(np.unique(y))
        
        # Step 1: Check the stopping criteria
        if (
            depth>self.max_depth or 
            n_labels==1 or 
            n_sample<self.min_sample_split
        ):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        

        # Step 2: Find the best split
        feat_idx = np.random.choice(n_feats, self.n_features, replace=False)
        
        best_feature, best_thresh = self._best_split(X, y, feat_idx)
        
        # Step 3: Create child nodes
        left_idxs, right_idxs = self._split(X[:,best_feature], best_thresh)
        left = self._build_tree(X[left_idxs,:], y[left_idxs], depth+1)
        right = self._build_tree(X[right_idxs,:], y[right_idxs], depth+1)
        
        return Node(best_feature, best_thresh, left,right) 
        
        
    def _best_split(self, X, y, feat_idxs):
        
        best_gain = -1
        split_idx, split_threshold = None, None
        
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            
            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
                    
        return split_idx, split_threshold
                    
    
    def _information_gain(self, y, X_column, thr):
        # Parents Entropy
        parent_entropy = self._entropy(y)
        # Children Entropy
        left_idx, right_idx = self._split(X_column, thr)
        
        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0
    
        # Calculate the Weighted Avg Entropy of Children
        n = len(y)
        n_l, n_r = len(left_idx), len(right_idx)
        e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r
        
        # Calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain
        
        
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column<= split_thresh).flatten()
        right_idxs = np.argwhere(X_column> split_thresh).flatten()
        return left_idxs, right_idxs
        
    def _entropy(self,y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p*np.log(p) for p in ps if p>0])
    
        
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x,node.left)
        return self._traverse_tree(x,node.right)

In [41]:
from sklearn import datasets

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [44]:
clf = DecisionTreeClassifier()

In [45]:
clf.fit(X_train,y_train)

In [46]:
prediction = clf.predict(X_test)

In [49]:
print(prediction)
print(y_test)

[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1
 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0
 1 1 0]
[1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0
 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 0]


In [50]:
def accuracy(y_pred, y_true):
    return sum(y_pred==y_true)/len(y_pred)

In [51]:
accuracy(prediction, y_test)

0.9298245614035088